![](https://mcd.unison.mx/wp-content/themes/awaken/img/logo_mcd.png)

# Curso Ingeniería de Características

## Análisis en Componentes Principales

### Julio Waissman


En esta libreta vamos a hacer una pequeña aplicación del Análisis en Componentes Principales (PCA) aplicado a los datos de marginación del estado de Sonora, los cuales los vamos a consultar [desde la página del gobierno federal para el año 2020](https://www.gob.mx/conapo/documentos/indices-de-marginacion-2020-284372).

No vamos a utilizar los indices normalizados de marginación por municipio desarrollados por la CONAPO ya que lo que queremos es hacer un análisis exploratorio de datos multivariados utilizndo componentes principales.

### 1. Cargando y seleccionando los datos de marginación

Empecemos por cargar los datos y ver como son:

In [ ]:
import pandas as pd

url = "http://www.conapo.gob.mx/work/models/CONAPO/Marginacion/Datos_Abiertos/Municipio/IMM_2020.xls"

dic_df = pd.read_excel(url, None)

dic_df.keys()

Nos vamos a quedar con sólo los municipios de sonora. En `IMM_2020` se encuentran los datos para fácil consumo de un programa, y en 'Diccionario' se encuentra la descripción de cada variable.

Vamos a ver en el diccionario de datos cuales son los datos que nos interesan.

In [ ]:
dic_df['Diccionario']

Vamos primero a separar los datos de sonora y revisar si se encuentran todos losmunicipios del estado:

In [ ]:
sonora = dic_df['IMM_2020'][dic_df['IMM_2020'].NOM_ENT.str.contains('Sonora')]
print(f"Los municipios de Sonora son: \n {sonora.NOM_MUN.unique()}")
sonora

Como podemos ver tenemos los 72 municipio. Nos vamos entonces a quedar con los datos de las columnas 7 a 15 (porcentajes) sin tomar en cuenta el tamaño de la población (en forma conciente)

In [ ]:
dic_datos = dic_df['Diccionario'].iloc[7:15,:].copy().rename(columns={
    "Indicadores sociodemográficos e índice de marginación a nivel municipal, 2020": "Indicador",
    "Unnamed: 1": "Descripcion" 
})

datos = sonora[dic_datos.Indicador.values].to_numpy()

# Vamos a ver, deberíamos tener 72 municipios, cada uno representado por 8 indicadores
datos.shape

# Y los indicadores que vamos a usar
print(f"Las variables a usar son: \n{dic_datos}")

### 2. Aplicando el Análisis en componentes principales

Vamos a ver si podemos utilizar PCA para revisar estos datos que se encuentran en 8 dimensiones y ver si los datos de marginación tendrían un significado diferente si se analizan a nivel estatal. Así no se tendrían los sesgos de municipios con menor marginación como Garza García o San Nicolás de los Garza en Nuevo León, Cuauthemoc o Benito Juárez en la Ciudad de México, pero tampoco de los municipios con mayor marginación como Batopilas en Chihuahua, Mezquital en Durango, Del Nayar en Nayarit o Mezquitic en Jalisco.  

In [ ]:
from sklearn.decomposition import PCA

n_components = 4

pca = PCA(n_components=n_components)
pca.fit(datos)

pd.DataFrame(
    np.transpose(pca.components_), 
    columns=[f'PC {x}' for x in range(1,n_components+1)],
    index = dic_datos.Indicador
)

¿Y cuanto explica sobre la varianza total cada una de las componentes? 

In [ ]:
import numpy as np
import plotly.express as px

print(pca.explained_variance_)

df_ev = pd.DataFrame({
    "acc": np.cumsum(pca.explained_variance_ratio_),
    "label": [f'PC {x}' for x in range(1,n_components+1)]
})
fig = px.scatter(df_ev, x='label', y='acc')
fig.show()




Como vemos, más del 0.93 % de la varianza se explica con la componente principal, la cual se compone principalmente de la variable `PL.5000`, que es el porcentaje de la población que vive en poblaciones menores a 5000 habitantes. Veamos como se comporta estavariable:

In [ ]:
sonora.columns

In [ ]:
fig = px.bar(sonora, x='CVE_MUN', y='PL.5000', text='NOM_MUN', color="PL.5000")
fig.show()

Y como lo vemos, pues la mayoría de los municipios son principalmente rurales. Separemos ahora los municipios con un `PL.5000` menor a 40 y revisemos:

In [ ]:
sonora_urbano = sonora[sonora['PL.5000'] < 40]
datos_urbano = sonora_urbano[dic_datos.Indicador.values].to_numpy()

pca_urbano = PCA(n_components=n_components)
pca_urbano.fit(datos_urbano)

df_ev = pd.DataFrame({
    "acc": np.cumsum(pca_urbano.explained_variance_ratio_),
    "label": [f'PC {x}' for x in range(1,n_components+1)]
})
fig = px.scatter(df_ev, x='label', y='acc')
fig.show()

pd.DataFrame(
    np.transpose(pca_urbano.components_), 
    columns=[f'PC {x}' for x in range(1,n_components+1)],
    index = dic_datos.Indicador
)

Y ahora veamos los municipios que llamaremos *rurales*

In [ ]:
sonora_rural = sonora[sonora['PL.5000'] > 40]
datos_rural = sonora_rural[dic_datos.Indicador.values].to_numpy()

pca_rural = PCA(n_components=n_components)
pca_rural.fit(datos_rural)

df_ev = pd.DataFrame({
    "acc": np.cumsum(pca_rural.explained_variance_ratio_),
    "label": [f'PC {x}' for x in range(1,n_components+1)]
})
fig = px.scatter(df_ev, x='label', y='acc')
fig.show()

pd.DataFrame(
    np.transpose(pca_rural.components_), 
    columns=[f'PC {x}' for x in range(1,n_components+1)],
    index = dic_datos.Indicador
)

### 3. Visualización de los componentes principales

Vamos a ver si hay similaridades entre municipios utilizando los 4 componentes principales. Para esto vamos a utilizar las gráficas de `scatter` que nos permite visualizar la relación de hasta 4 variables diferentes.

Vamos a hacer el ejercicio con los datos de `sonora_urbano` y dejamos el análisis de los municipios rurales como ejercicio.

In [ ]:
df_urbano = pd.DataFrame(
    pca_urbano.transform(datos_urbano),
    columns= [f'PC {x}' for x in range(1,n_components+1)],
    index= sonora_urbano.NOM_MUN    
)

fig = px.scatter(df_urbano, x='PC 1', y='PC 2', color='PC 3', text=df_urbano.index)
fig.show()